In [1]:
import numpy as np 
import theano 
import theano.tensor as T 

### theano.function

In [8]:
x = T.dmatrix('x')

s = 1 / (1+ T.exp(x))  # logistic soft step

logistic = theano.function(
    inputs=[x], # Input variables of a Theano function 規定要放在list內
    outputs=[s] # outputs 若只有一個，則強制要放在list內，有放的話表示會放在list內輸出
)

print(logistic([[0, 1], [-2, -3]]))

[array([[0.5       , 0.26894142],
       [0.88079708, 0.95257413]])]


In [16]:
a, b = T.dmatrices('a', 'b')
diff = a -b
abs_diff = abs(diff)
diff_square = diff**2

f = theano.function(
    [a, b], 
    [diff, abs_diff, diff_square]
)

x1, x2, x3 = f(np.ones((2, 2)), np.arange(4).reshape(2,2))
print(x1, x2, x3)

[[ 1.  0.]
 [-1. -2.]] [[1. 0.]
 [1. 2.]] [[1. 0.]
 [1. 4.]]


### theano.scan

In [42]:
## 範例一

x = T.vector('x')

def square(h):
    return h*h

outputs, update = theano.scan(    # 這裡的output就是一個用list包起來的數組(sequence)
    fn=square, 
    sequences=x, 
    n_steps=2  # 輸入2就只會run 前兩個element, 不指定就是 run 每個sequence的所有元素
)

square_op = theano.function(
    inputs=[x], 
    outputs=[outputs],          # output is a array list 這裡再用一個list包一層
)

o_val = square_op(np.array([1,2,3,4,5]))

print(o_val)

[array([1., 4.])]


In [2]:
## 範例二 Fibonacci
N = T.iscalars('N')

def recurrence(n, fn_1, fn_2):
    return fn_1+fn_2, fn_1

outputs, _ = theano.scan(  # recurrence的 return有兩個參數，經過scan跑出來會是一個list內部包著兩個array list
    fn=recurrence,
    sequences=T.arange(N), 
    outputs_info=[1., 1.],
    n_steps=N,
)

fabonacci = theano.function(
    inputs=[N], 
    outputs= outputs, 
)

o_val = fabonacci(8)

print(o_val)


NameError: name 'T' is not defined

In [26]:
# 做實驗 針對 srn_parity.py的疑問
import numpy as np 
import theano 
import theano.tensor as T

W = theano.shared(np.random.randn(3, 3))
thA_t = T.matrix('A')
def mutiple(a_t):
    y_t = a_t.dot(W)
    return y_t, y_t*2

[p1, p2], _ = theano.scan(
    fn=mutiple, 
    sequences=thA_t, 
    n_steps=thA_t.shape[0]
)

op = theano.function(
    inputs=[thA_t], 
    outputs=p1
)

A_t = np.arange(9).reshape(3, 3)
o = op(A_t)
print(o)

[[ -2.19151694   1.33328589   1.0556425 ]
 [-10.72374481   2.57853563   1.0224578 ]
 [-19.25597267   3.82378536   0.98927309]]
